## siRNA knockdown calibrated ##

This notebook fits analytical functions to Rafał’s data.

## Motivation
siRNA (small interfering RNA) triggers specific degradation of mRNA. The RISC (RNA-induced silencing complex), which consists of siRNA and some proteins, cuts mRNA containing a strand sequence complementary to the sequence of the siRNA. Reducing protein expression by adding siRNA is called gene knockdown.

Gene knockdown is a promising approach for the treatment of some diseases, e.g. cancer. The aim of this project is to study the influence of siRNA on gene expression and mRNA degradation on the single cell level to promote development of siRNA-based medical treatments.

This is done by fitting the solutions of the differential equations describing the expression network to measured fluorescence traces of cells transfected with a GFP mRNA and a RFP mRNA, where siRNA specific for GFP mRNA is added and RFP is used as a reference.

Among the fit parameters, there is the GFP mRNA degradation rate $\delta_\text{g}$ and the RFP mRNA degradation rate $\delta_\text{r}$.

The solutions look like this:

$$f_\text{red}(t) =
m_\text{r}\,k_\text{tl} \left(
\frac{1}{\beta_\text{r}-\delta_\text{r}+k_\text{m,r}} \mathrm{e}^{-(\beta_\text{r}+k_\text{m,r})(t-t_0)}
-\frac{1}{\beta_\text{r} - \delta_\text{r}} \mathrm{e}^{-\beta_\text{r} (t-t_0)}
+\frac{k_\text{m,r}}{(\beta_\text{r}-\delta_\text{r}) (\beta_\text{r}-\delta_\text{r}+k_\text{m,r})} \mathrm{e}^{-\delta_\text{r} (t-t_0)}
\right)
$$

$$f_\text{green}(t) =
m_\text{g}\,k_\text{tl} \left(
\frac{1}{\beta_\text{g}-\delta_\text{g}+k_\text{m,g}} \mathrm{e}^{-(\beta_\text{g}+k_\text{m,g})(t-t_0)}
-\frac{1}{\beta_\text{g} - \delta_\text{g}} \mathrm{e}^{-\beta_\text{g} (t-t_0)}
+\frac{k_\text{m,g}}{(\beta_\text{g}-\delta_\text{g}) (\beta_\text{g}-\delta_\text{g}+k_\text{m,g})} \mathrm{e}^{-\delta_\text{g} (t-t_0)}
\right)
$$

## Notebook structure
The notebook has the following structure:

At first, the model functions are defined and the data is loaded. The next section contains code for fitting the two models separately. The next section contains code for fitting the two traces in one run with parameters shared among the models.

Fitting requires that the result list `R` is defined, which can be done by running the corresponding cell. When `R` has been populated by fitting, the results can be plotted. There are cells for plotting the results of the separate fit, the results of the combined fit, and the pure parameter distributions of all fits.

Additionally, there are cells for saving and loading paramaters by python’s `pickle` module.

In [ ]:
# Import modules needed
%matplotlib inline
import numpy as np
np.seterr(divide='print')
import scipy as sc
import lmfit as lm
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.gridspec import GridSpec
import pandas as pd
from io_Daniel import *
import os
import sys
import inspect
import pickle
import ipywidgets as wdg
import IPython
from collections import OrderedDict
from sklearn.neighbors import KernelDensity

In [ ]:
def red(t, tr, m_ktl, kmr, betr, deltr, offr):
    """Model function for red data"""

    f = np.zeros(np.shape(t))
    idx_after = (t > tr)
    dt = t[idx_after] - tr

    f1 = np.exp(- (betr + kmr) * dt) / (betr - deltr + kmr)
    f2 = - np.exp(- betr * dt) / (betr - deltr)
    f3 = kmr * np.exp(- deltr * dt) / (betr - deltr) / (betr - deltr + kmr)

    f[idx_after] = (f1 + f2 + f3) * m_ktl

    return f + offr

In [ ]:
def green(t, tg, m_ktl, kmg, betg, deltg, offg):
    """Model function for green data"""

    f = np.zeros(np.shape(t))
    idx_after = t > tg
    dt = t[idx_after] - tg

    f1 = np.exp(- (betg + kmg) * dt) / (betg - deltg + kmg)
    f2 = - np.exp(- betg * dt) / (betg - deltg)
    f3 = kmg * np.exp(- deltg * dt) / (betg - deltg) / (betg - deltg + kmg)

    f[idx_after] = (f1 + f2 + f3) * m_ktl

    return f + offg

In [ ]:
def combined(t, tr, tg, m_ktl, kmr, kmg, betr, betg, deltr, deltg, offr, offg):
    """Model function for a combined fit of red and green data"""

    f = np.stack(
        (red(t=t, tr=tr, m_ktl=m_ktl, kmr=kmr, betr=betr, deltr=deltr, offr=offr),
         green(t=t, tg=tg, m_ktl=m_ktl, kmg=kmg, betg=betg, deltg=deltg, offg=offg)),
        axis=1)

    return f

In [ ]:
# Set default parameter values
m_ktl_0 = 2e4

tr_0 = 4.5
kmr_0 = 0.1
betr_0 = 0.3
deltr_0 = 0.03
offr_0 = 0

tg_0 = 2
kmg_0 = 0.1
betg_0 = 0.04
deltg_0 = 2
offg_0 = 0

MAX_m_ktl = 5e5
MAX_tr = 30
MAX_tg = 30
MAX_kmr = 30
MAX_kmg = 30
MAX_betr = 10
MAX_betg = 10
MAX_deltr = 11
MAX_deltg = 11

MIN_m_ktl = 1

In [ ]:
class FitParameters:
    """FitParameters facilitates managing values and bounds of fit parameters"""
    def __init__(self, fun, independent=[], fixed=[]):
        # Store function
        self.fun = fun

        # Get parameters of fun
        params = inspect.signature(self.fun).parameters

        # Build data frame of parameters
        self.df = pd.DataFrame(columns=['value', 'min', 'max'],
                               index=[p for p in params.keys()],
                               dtype=np.float64)

        # Set “independent” and “fixed” flag
        self.df.add(pd.DataFrame(columns=['independent', 'fixed'], dtype=np.bool))
        for p in self.df.index.values:
            self.df.loc[p, 'independent'] = p in independent
            self.df.loc[p, 'fixed'] = p in fixed

        # Set default parameters
        for p in self.df.index.values:
            if params[p].default == inspect.Parameter.empty:
                if self.df.loc[p, 'independent']:
                    self.df.loc[p, 'value'] = np.NaN
                else:
                    self.df.loc[p, 'value'] = 0
            else:
                self.df.loc[p, 'value'] = params[p].default

    def set(self, p, **props):
        """Allows user to change parameter properties"""
        if p not in self.df.index.values:
            raise KeyError("Unknown parameter name: {}".format(par))

        for prop, val in props.items():
            if prop == 'value':
                self.df.loc[p, 'value'] = val
            elif prop == 'min':
                self.df.loc[p, 'min'] = val
            elif prop == 'max':
                self.df.loc[p, 'max'] = val
            elif prop == 'independent':
                self.df.loc[p, 'independent'] = val
            elif prop == 'fixed':
                self.df.loc[p, 'fixed'] = val
            else:
                raise KeyError("Illegal parameter property: {}".format(prop))

    def eval_params(self, params=[], **vals):
        """Returns parameters for evaluating the function.

        Arguments:
        params: optional list of values of free parameters
        vals: dictionary of parameter values

        If a value for a parameter is specified in both `params` and `vals`,
        the value from `vals` is used.
        Values for independent parameters must be specified in `vals`."""
        # Add additional values from `params` to vals
        if len(params) != 0:
            par_names = self.names()
            if np.size(par_names) != len(params):
                raise ValueError("Wrong number of parameters given ({})".format(len(params)))
            for pn, pv in zip(par_names, params):
                if pn not in vals:
                    vals[pn] = pv

        # Fill values unspecified so far from `self.df`
        for p in self.df.index.values:
            if p not in vals:
                if self.df.loc[p, 'independent']:
                    raise ValueError("Independent parameter `{}` not specified".format(p))
                else:
                    vals[p] = self.df.loc[p, 'value']
        return vals

    def eval(self, params=[], **vals):
        """Evaluates the function.

        Arguments:
        params: optional list of values of free parameters
        vals: dictionary of parameter values

        If a value for a parameter is specified in both `params` and `vals`,
        the value from `vals` is used.
        Values for independent parameters must be specified in `vals`."""
        return self.fun(**self.eval_params(params, **vals))

    def freeIdx(self):
        """Returns a list of names of free parameters"""
        return [p for p in self.df.index.values
                if not (self.df.loc[p, 'independent'] or self.df.loc[p, 'fixed'])]

    def bounds(self):
        """Returns a list of bound tuples of free parameters
        for use in scipy.optimize.minimize"""
        bnds = []
        for p in self.freeIdx():
            # Get parameter bounds
            min_val = self.df.loc[p, 'min']
            max_val = self.df.loc[p, 'max']

            # Replace missing values with default minimum and maximum values
            if np.isnan(min_val):
                min_val = None
            if np.isnan(max_val):
                max_val = None

            # Append to bounds list
            bnds.append((min_val, max_val))
        return bnds

    def initial(self):
        """Returns a numpy.ndarray of initial values for use in scipy.optimize.minimize"""
        return self.df.loc[self.freeIdx(), 'value'].values.copy()

    def index(self, p):
        """Returns the index of a given parameter in the parameter vector"""
        idx = np.flatnonzero(self.df.index.values == p)
        if len(idx) == 0:
            raise KeyError("Unknown parameter name: {}".format(p))
        return idx[0]

    def names(self, onlyFree=True):
        """Returns an array of the parameter names.

        If `onlyFree == True`, only free parameters are returned.
        Else, all parameters (including independent and fixed parameters) are returned."""
        if onlyFree:
            return np.array(self.freeIdx(), dtype=np.object_)
        else:
            return self.df.index.values.copy()

In [ ]:
# Alternative separate models for scipy.optimize.minimize
red_p = FitParameters(red, independent='t')
red_p.set('tr', min=0, max=MAX_tr, value=tr_0)
red_p.set('m_ktl', min=MIN_m_ktl, max=MAX_m_ktl, value=m_ktl_0)
red_p.set('kmr', min=0, max=MAX_kmr, value=kmr_0)
red_p.set('betr', min=0.001, max=MAX_betr, value=betr_0)
red_p.set('deltr', min=0.001, max=MAX_deltr, value=deltr_0)
red_p.set('offr', value=offr_0)

green_p = FitParameters(green, independent='t')
green_p.set('tg', min=0, max=MAX_tg, value=tg_0)
green_p.set('m_ktl', min=MIN_m_ktl, max=MAX_m_ktl, value=m_ktl_0)
green_p.set('kmg', min=0, max=MAX_kmg, value=kmg_0)
green_p.set('betg', min=0.001, max=MAX_betg, value=betg_0)
green_p.set('deltg', min=0.001, max=MAX_deltg, value=deltg_0)
green_p.set('offg', value=offg_0)

In [ ]:
# Alternative combined model for scipy.optimize.minimize
combined_p = FitParameters(combined, independent = 't')
combined_p.set('tr', min=0, max=MAX_tr, value=tr_0)
combined_p.set('tg', min=0, max=MAX_tg, value=tg_0)
combined_p.set('m_ktl', min=MIN_m_ktl, max=MAX_m_ktl, value=m_ktl_0)
combined_p.set('kmr', min=0, max=MAX_kmr, value=kmr_0)
combined_p.set('kmg', min=0, max=MAX_kmg, value=kmg_0)
combined_p.set('betr', min=0, max=MAX_betr, value=betr_0)
combined_p.set('betg', min=0, max=MAX_betg, value=betg_0)
combined_p.set('deltr', min=0, max=MAX_deltr, value=deltr_0)
combined_p.set('deltg', min=0, max=MAX_deltg, value=deltg_0)
combined_p.set('offr', value=offr_0)
combined_p.set('offg', value=offg_0)

## Jacobian
To increase the efficiency of fitting, the Jacobian matrix of the objective function is provided to the optimization routine.
If the objective function is a typical negative log-likelihood function with normal distribution of residuals
$$
 L(\theta) = \sum_{t\in T} \frac{1}{2\sigma_t^2} \big(D_t - f(t\mid\theta)\big)^2 \text{,}
$$
where $D_t$ is the measured data at time $t$ and $f(t\mid\theta)$ is the value of the model function at time $t$ with parameters $\theta$, the Jacobian is:
$$\begin{align}
\nabla L(\theta) &= \nabla \sum_{t\in T} \frac{1}{2\sigma_t^2} \big(D_t - f\left(t\,\middle|\,\theta\right)\big)^2 \\
&= \sum_{t\in T} \nabla \frac{1}{2\sigma_t^2} \big( D_t - f\left(t\,\middle|\,\theta\right) \big)^2 \\
&= \sum_{t\in T} \frac{1}{\sigma_t^2} \big( D_t - f\left(t\,\middle|\,\theta\right) \big) \nabla\big( D_t - f\left(t\,\middle|\,\theta\right) \big) \\
&= \sum_{t\in T} \frac{1}{\sigma_t^2} \big( D_t - f\left(t\,\middle|\,\theta\right) \big)\big(\nabla D_t - \nabla  f\left(t\,\middle|\,\theta\right)\big) \\
&= -\sum_{t\in T} \frac{1}{\sigma_t^2} \big(D_t - f\left(t\,\middle|\,\theta\right)\big) \nabla f\left(t\,\middle|\,\theta\right) \\
\end{align}$$
We see that for calculating the Jacobian of the objective function we need the Jacobian of the model function.

We use the general expression model function
$$
f\left(t \,\middle|\, t_0, m, k, \beta, \delta, a\right) = a + m \left(\frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)} + \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{k + \beta - \delta} - \frac{\mathrm{e}^{- \beta \left(t - t_{0}\right)}}{\beta - \delta}\right) \text{,}
$$
where $t_0$ is the mRNA expression onset time, $m$ is the product of initial mRNA amount and translation rate, $k$ is the maturation rate, $\beta$ is the protein degradation rate, $\delta$ is the mRNA degradation rate, and $a$ is a vertical offset.

The Jacobian $\nabla f\left(t \,\middle|\, t_0, m, k, \beta, \delta, a\right)$ of the general expression model function is the vector of the derivatives with respect to the various parameters:
$$\begin{align*}
\frac{\partial f}{\partial t_0} &= m \left(\frac{k \delta \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)} - \frac{\beta \mathrm{e}^{- \beta \left(t - t_{0}\right)}}{\beta - \delta} + \frac{\left(k + \beta\right) \mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{k + \beta - \delta}\right)\\
\frac{\partial f}{\partial m} &= \frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)} + \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{k + \beta - \delta} - \frac{\mathrm{e}^{- \beta \left(t - t_{0}\right)}}{\beta - \delta}\\
\frac{\partial f}{\partial k} &= m \left(- \frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)^{2}} + \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{k + \beta - \delta} \left(- t + t_{0}\right) - \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{\left(k + \beta - \delta\right)^{2}} + \frac{\mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)}\right)\\
\frac{\partial f}{\partial \beta} &= m \left(- \frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)^{2}} - \frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right)^{2} \left(k + \beta - \delta\right)} + \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{k + \beta - \delta} \left(- t + t_{0}\right) - \frac{\mathrm{e}^{- \beta \left(t - t_{0}\right)}}{\beta - \delta} \left(- t + t_{0}\right) - \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{\left(k + \beta - \delta\right)^{2}} + \frac{\mathrm{e}^{- \beta \left(t - t_{0}\right)}}{\left(\beta - \delta\right)^{2}}\right)\\
\frac{\partial f}{\partial \delta} &= m \left(\frac{k \left(- t + t_{0}\right) \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)} + \frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right) \left(k + \beta - \delta\right)^{2}} + \frac{k \mathrm{e}^{- \delta \left(t - t_{0}\right)}}{\left(\beta - \delta\right)^{2} \left(k + \beta - \delta\right)} + \frac{\mathrm{e}^{\left(- k - \beta\right) \left(t - t_{0}\right)}}{\left(k + \beta - \delta\right)^{2}} - \frac{\mathrm{e}^{- \beta \left(t - t_{0}\right)}}{\left(\beta - \delta\right)^{2}}\right)\\
\frac{\partial f}{\partial a} &= 1\\
\end{align*}$$

In [ ]:
def general_jacobian(**params):
    
    a = params['offset']
    t0 = params['t0']
    

## Read in data and prepare result list

In [ ]:
# Calculate kernel density estimation of parameter distributions
def parameter_KDE(par_tab):
    """Returns a kernel density estimation of the parameter values for plotting"""
    dens_res = 200
    bw_div = 15

    par_dist = {}

    for par_name in par_tab.columns:
        # Get parameter values
        par_vals = par_tab.loc[:,par_name].values
        par_vals = par_vals.reshape((par_vals.size, 1))

        # Test parameter values for validity
        if np.any(np.logical_not(np.isfinite(par_vals))):
            print("Warning: invalid values encountered for “{}”".format(par_name))
            par_vals = par_vals(np.isfinite(par_vals))
            if par_vals.size > 0:
                # Calculate distribution of valid entries
                par_vals = par_vals.reshape((par_vals.size, 1))
            else:
                # No valid entries found; cancel distribution calculation
                par_dist[par_name] = {'val': [], 'prob': []}
                continue

        # Get parameter extrema and bandwidth
        par_min = np.min(par_vals)
        par_max = np.max(par_vals)
        bw = (par_max - par_min) / bw_div

        # Get kernel density estimation of parameter values
        kde = KernelDensity(kernel='epanechnikov', bandwidth=bw).fit(par_vals)
        par_x = np.linspace(par_min, par_max, dens_res).reshape((dens_res, 1))
        par_dens = np.exp(kde.score_samples(par_x))

        # Adjust values for nicer plotting (KDE >= 0, edges == 0)
        #par_dens[par_dens < 0] = 0
        if par_dens[0] != 0:
            par_dens = np.insert(par_dens, 0, 0)
            par_x = np.insert(par_x, 0, par_min)
        if par_dens[-1] != 0:
            par_dens = np.append(par_dens, 0)
            par_x = np.append(par_x, par_max)

        # Insert KDE into dict
        par_dist[par_name] = {'val': par_x.flatten(), 'prob': par_dens.flatten()}
    return par_dist

In [ ]:
def plot_kde(ax, dist, label, clr_face='b', clr_edge='k', mark=None):
    """Plots the current parameter value in relation to the distribution
    in the whole dataset."""
    ax.fill_betweenx(dist['val'], dist['prob'], color=clr_face)
    if mark != None:
        ax.axhline(y=mark, color=clr_edge)
    ax.set_xticks([])
    ax.spines['left'].set_position('zero')
    for s in [ax.spines[pos] for pos in ['bottom', 'right', 'top']]:
        s.set_visible(False)
    ax.set_title(label)
    #ax.get_yaxis().set_major_formatter(StrMethodFormatter('{x:.2g}'))

In [ ]:
# Prepare data loading

# Define available files
datafiles = [
    {
        "sample": "A549",
        "condition": "control",
        "measurement": "Test",
        "file": "data/A549_control_test.xlsx"
    },
    {
        "sample": "A549",
        "condition": "siRNA",
        "measurement": "2016-01-09_seq3",
        "file": "data/2016-01-09_seq3_A549_siRNA_#molecules.xlsx"
    }, {
        "sample": "A549",
        "condition": "control",
        "measurement": "2016-01-09_seq5",
        "file": "data/2016-01-09_seq5_A549_Control_#molecules.xlsx"
    }, {
        "sample": "A549",
        "condition": "siRNA",
        "measurement": "2016-12-20_seq3",
        "file": "data/2016-12-20_seq3_A549_siRNA_#molecules.xlsx"
    }, {
        "sample": "A549",
        "condition": "control",
        "measurement": "2016-12-20_seq4",
        "file": "data/2016-12-20_seq4_A549_control_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "siRNA",
        "measurement": "2017-05-26_seq10",
        "file": "data/2017-05-26_seq10_Huh7_siRNA_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "siRNA",
        "measurement": "2017-05-26_seq11",
        "file": "data/2017-05-26_seq11_Huh7_siRNA_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "control",
        "measurement": "2017-05-26_seq6",
        "file": "data/2017-05-26_seq6_Huh7_control_#molecules.xlsx"
    }, {
        "sample": "Huh7",
        "condition": "control",
        "measurement": "2017-05-26_seq7",
        "file": "data/2017-05-26_seq7_Huh7_control_#molecules.xlsx"
    }
]

# By default, mark all files for loading
load_idcs = range(len(datafiles))

# Define function for loading data
def load_data_from_files():
    """Loads data from specified files into `D`.
    Requires `load_idcs` to hold a list of indices to `datafiles`."""
    global D
    D = []
    for i in load_idcs:
        # Show message
        print("Loading file: {}".format(datafiles[i]["file"]))

        # Read sheets from excel file
        X = pd.read_excel(datafiles[i]['file'], dtype=np.float64, sheetname=[
            '#RFP', '#GFP_corrected', '#RFP_error', '#GFP_error'])

        # Write data into easy-to-access structure
        d = {}
        d['sample'] = datafiles[i]['sample']
        d['condition'] = datafiles[i]['condition']
        d['measurement'] = datafiles[i]['measurement']
        d['file'] = datafiles[i]['file']
        d['t'] = X['#RFP'].values[:,0].flatten()
        #d['rfp'] = X['RFP'].values[:,1:]
        #d['gfp'] = X['GFP_corrected'].values[:,1:]
        d['rfp'] = X['#RFP'].values[:,1:]
        d['gfp'] = X['#GFP_corrected'].values[:,1:]
        d['rfp_error'] = X['#RFP_error'].values[:,1:]
        d['gfp_error'] = X['#GFP_error'].values[:,1:]
        D.append(d)

In [ ]:
def getDataLabel(i, filename=False):
    """Returns a nicely formatted name for the `i`-th element of `D`.
    Set `filename=True` for a filename-friendly output."""
    if filename:
        return "{0[measurement]}_{0[sample]}_{0[condition]}".format(D[i])
    return "{0[sample]}: {0[condition]} [{0[measurement]}]".format(D[i])

In [ ]:
# Read in data from excel sheets

# Prompt user for files to load
lbl = wdg.Label('Select the files to load:')
lbl.layout.width = 'initial'
entries = []
for f in datafiles:
    entries.append("{} {}: {}".format(
        f['sample'], f['condition'], f['file']))
sel_entry = wdg.SelectMultiple(options=entries, rows=len(entries))
sel_entry.layout.width = 'initial'
bload = wdg.Button(description='Load')
bselall = wdg.Button(description='Select all')
bselnone = wdg.Button(description='Select none')

# Define callbacks
def sel_all_files(_):
    sel_entry.value = entries
def sel_no_files(_):
    sel_entry.value = ()
def load_button_clicked(_):
    global load_idcs
    load_idcs = [entries.index(r) for r in sel_entry.value]
    vb.close()
    load_data_from_files()
bselall.on_click(sel_all_files)
bselnone.on_click(sel_no_files)
bload.on_click(load_button_clicked)

# Finally, show the widgets
vb = wdg.VBox((lbl, sel_entry, wdg.HBox((bload,bselall,bselnone))))
IPython.display.display(vb)

In [ ]:
# Provide output tables

# Initialize result dictionary
R = []

# Get a list of fit parameters
par_names = green_p.names().tolist()
par_names.extend(p for p in red_p.names() if p not in par_names)
par_names.sort()

# Iteratively populate the result dictionary
for k in range(len(D)):
    R.insert(k, {})
    nTraces = np.shape(D[k]['gfp'])[1]
    nTimes = np.shape(D[k]['gfp'])[0]
    tpl_traces = np.empty((nTimes, nTraces))
    tpl_traces.fill(np.NaN)

    R[k]['green'] = {}
    R[k]['green']['params'] = pd.DataFrame(index=np.arange(nTraces), columns=green_p.names(), dtype='float64')
    R[k]['green']['fit'] = np.copy(tpl_traces)

    R[k]['red'] = {}
    R[k]['red']['params'] = pd.DataFrame(index=np.arange(nTraces), columns=red_p.names(), dtype='float64')
    R[k]['red']['fit'] = np.copy(tpl_traces)

    R[k]['combined'] = {}
    R[k]['combined']['params'] = pd.DataFrame(index=np.arange(nTraces), columns=combined_p.names(), dtype='float64')
    #R[k]['combined']['fit'] = {}

### Pickle or load fitting results
Pickling is only reasonable if the result list `R` has already been populated by fitting (see below).

In [ ]:
# Pickle fit results for future sessions
outfile = getTimeStamp() + '_fit_results.pickled'
with open(outfile, 'wb') as f:
    pickle.dump(R, f)

In [ ]:
# Load pickled results (requires file suffix “.pickled”)
pickfiles = [f for f in os.listdir() if f.lower().endswith('.pickled')]
pickfiles.sort(reverse=True)

lbl = wdg.Label('Select the file to load:')
lbl.layout.width = 'initial'
rad = wdg.RadioButtons(options=pickfiles)
but = wdg.Button(description='Load')
vb = wdg.VBox([lbl, rad, but])
IPython.display.display(vb)

def clicked_on_but(b):
    global R
    with open(rad.value, 'rb') as f:
        R = pickle.load(f)
    print('Loaded: ' + rad.value)
    vb.close()
but.on_click(clicked_on_but)

## Fit and plot separate models

In [ ]:
def plotSeparate(ds, tr, pdf=None, par_kde=None):
    """Fits and plots the data, treating RFP and GFP separately.

    Keyword arguments:
    ds -- the dictionary key of the dataset
    tr -- the index of the trace in the dataset to be processed
    pdf -- a PdfPages object to which the figure is written if it is not None
    par_kde -- if containing dict of values of parameter distributions, plot distributions
    """

    # Plot fit results
    fig = plt.figure()

    if par_kde != None:
        fig.set_figwidth(1.6 * fig.get_figwidth())

        pn_red = ['m_ktl', 'tr', 'kmr', 'betr', 'deltr', 'offr']
        pn_green = ['m_ktl', 'tg', 'kmg', 'betg', 'deltg', 'offg']

        grid = (2, max(len(pn_red), len(pn_green)))
        gs = GridSpec(grid[0], grid[1])

        # Plot green parameters
        for pi, label in enumerate(pn_green):
            ax = plt.subplot(gs.new_subplotspec((0, pi)))
            data = par_kde['green'][label]
            clr_face = '#00ff0055'
            clr_edge = '#009900ff'
            curr_val = R[ds]['green']['params'].loc[tr,label]
            plot_kde(ax, data, label, clr_face, clr_edge, curr_val)

        # Plot red parameters
        for pi, label in enumerate(pn_red):
            ax = plt.subplot(gs.new_subplotspec((1, pi)))
            data = par_kde['red'][label]
            clr_face = '#ff000055'
            clr_edge = '#990000ff'
            curr_val = R[ds]['red']['params'].loc[tr,label]
            plot_kde(ax, data, label, clr_face, clr_edge, curr_val)

        # Adjust subplot layout
        gs.tight_layout(fig, pad=0, rect=(0.5, 0, 1, 1))

        # Create axes for fit
        gs_fit = GridSpec(1, 1)
        ax = fig.add_subplot(gs_fit[0])
        gs_fit.tight_layout(fig, pad=0, rect=(0, 0, 0.5, 1))

    else:
        ax = fig.gca()

    p_tr = ax.axvline(R[ds]['red']['params']['tr'][tr], label='RFP onset',
                       color='#ff0000', linewidth=.5, linestyle='--')
    p_tg = ax.axvline(R[ds]['green']['params']['tg'][tr], label='GFP onset',
                      color='#00ff00', linewidth=.5, linestyle='--')
    p_fr, = ax.plot(D[ds]['t'], R[ds]['red']['fit'][:,tr], '-', label='RFP (fit)', color='#ff0000', linewidth=1)
    p_fg, = ax.plot(D[ds]['t'], R[ds]['green']['fit'][:,tr], '-', label='GFP (fit)', color='#00ff00', linewidth=1)
    p_dr, = ax.plot(D[ds]['t'], D[ds]['rfp'][:,tr], '-', label='RFP (measured)', color='#990000', linewidth=.5)
    p_dg, = ax.plot(D[ds]['t'], D[ds]['gfp'][:,tr], '-', label='GFP (measured)', color='#009900', linewidth=.5)

    # Format plot
    ax.set_xlabel('Time [h]')
    ax.set_ylabel('Fluorescence intensity [a.u.]')
    ax.set_title('{} #{:03d}\n(separate fit)'.format(getDataLabel(ds), tr))
    ax.legend(handles=[p_dg, p_fg, p_tg, p_dr, p_fr, p_tr])

    # Write figure to pdf
    if pdf != None:
        pdf.savefig(fig, bbox_inches='tight')

    # Show and close figure
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Fit traces separately
for ds in range(len(D)):
    nTraces = np.shape(D[ds]['rfp'])[1]

    for tr in range(nTraces):
        print('Fitting „{}“ #{:03d}/{:03d} …'.format(getDataLabel(ds), tr, nTraces))
        
        # Prepare data
        data_red = D[ds]['rfp'][:,tr].flatten()
        data_green = D[ds]['gfp'][:,tr].flatten()

        wght_red = D[ds]['rfp_error'][:,tr]**2
        wght_green = D[ds]['gfp_error'][:,tr]**2
        
        # Adjust parameter properties for onset time and offset
        red_p.set('tr', max=D[ds]['t'][data_red.argmax()])
        red_p.set('offr',
                       min=data_red[:10].min(),
                       max=data_red[:10].max(),
                       value=np.median(data_red[:10]))
        green_p.set('tg', max=D[ds]['t'][data_green.argmax()])
        green_p.set('offg',
                       min=data_green[:10].min(),
                       max=data_green[:10].max(),
                       value=np.median(data_green[:10]))

        # Objective function (closure)
        i_obj = 0
        isChisqRedNan = False
        def objective_fcn(params):
            """Objective function for separate model"""

            # Compute chisquare
            cur_val = red_p.eval(params, t=D[ds]['t'])
            chisq = np.sum(.5 * (data_red - cur_val)**2 / wght_red)

            # DEBUG
            global i_obj, isChisqRedNan
            #print("Trace {:03d}, i={:03d}: chisq={}".format(tr, i_obj, chisq))
            #for p,v in zip(red_p.names(), params):
            #    print("\t{:>10s} = {}".format(p, v))

            # Print model values at first iteration with NaN chisquare
            #if np.isnan(chisq) and not isChisqRedNan:
            #    print(cur_val)
            #    isChisqNan = True
            i_obj += 1

            return chisq

        # Fit the data
        result = sc.optimize.minimize(objective_fcn,
                                      red_p.initial(),
                                      method='TNC',# one of: 'SLSQP' 'TNC' 'L-BFGS-B'
                                      bounds=red_p.bounds(),
                                      options={'disp':True,
                                               'maxiter': 10000}
                                     )

        # Print result
        print("\tRed success {}: {}".format(result.success, result.message))

        # Save results to R
        R[ds]['red']['params'].iloc[tr] = result.x
        best_fit = red(D[ds]['t'], *result.x)
        R[ds]['red']['fit'][:,tr] = best_fit

        # Fit green data
        i_obj = 0
        isChisqGreenNan = False
        def objective_fcn(params):
            """Objective function for green model"""

            # Computer chisquare
            cur_val = green_p.eval(params, t=D[ds]['t'])
            chisq = np.sum(.5 * (data_green - cur_val)**2 / wght_green)

            # DEBUG
            global i_obj, isChisqGreenNan
            #print("Trace {:03d}, i={:03d}: chisq={}".format(tr, i_obj, chisq))
            #for p,v in zip(green_p.names(), params):
            #    print("\t{:>10s} = {}".format(p, v))

            # Print model values at first iteration with NaN chisquare
            if np.isnan(chisq) and not isChisqGreenNan:
            #    print(cur_val)
                isChisqNan = True
            i_obj += 1

            return chisq

        result = sc.optimize.minimize(objective_fcn,
                                      green_p.initial(),
                                      method='TNC',
                                      bounds=green_p.bounds(),
                                      options={'disp': True,
                                               'maxiter': 10000})
        print("\tGreen success {}: {}".format(result.success, result.message))

        R[ds]['green']['params'].iloc[tr] = result.x
        best_fit = green(D[ds]['t'], *result.x)
        R[ds]['green']['fit'][:,tr] = best_fit

        # DEBUG
        #if tr >= 2:
        #    print("Breaking loop for debugging purposes")
        #    break

In [ ]:
# Plot results of separate fit
ts = getTimeStamp()

for ds in range(len(D)):
    par_kde = {}
    for t in ('red', 'green'):
        par_kde[t] = parameter_KDE(R[ds][t]['params'])
    pdffile = os.path.join(getOutpath(), '{}_separate_{}.pdf'.format(ts, getDataLabel(ds, True)))
    with PdfPages(pdffile) as pdf:
        for tr in range(np.shape(D[ds]['rfp'])[1]):
            plotSeparate(ds, tr, pdf, par_kde)

            # DEBUG
            #if tr >= 2:
            #    print("Break loop")
            #    break

## Fit and plot combined model

In [ ]:
def plotCombined(ds, tr, pdf=None, par_kde=None):
    """Fits and plots the data, treating RFP and GFP together.
    
    Keyword arguments:
    ds -- the dictionary key of the dataset
    tr -- the index of the trace in the dataset to be processed
    pdf -- a PdfPages object to which the figure is written if it is not None
    par_kde -- if containing dict of values of parameter distributions, plot distributions
    """

    # Plot fit results
    fig = plt.figure()
    #fig.set_tight_layout(False)

    if par_kde != None:
        fig.set_figwidth(1.6 * fig.get_figwidth())

        #pn_both = ['m', 'ktl']
        pn_both = ['m_ktl']
        pn_red = ['tr', 'kmr', 'betr', 'deltr', 'offr']
        pn_green = ['tg', 'kmg', 'betg', 'deltg', 'offg']

        # Plot combined parameters
        grid = (2, 1+max(len(pn_red), len(pn_green)))
        gs = GridSpec(grid[0], grid[1])

        #for pi, label in enumerate(pn_both):
        pi = 0
        label = pn_both[pi]
        ax = plt.subplot(gs.new_subplotspec((pi, 0), rowspan=2))
        data = par_kde['combined'][label]
        clr_face = '#0000ff55'
        clr_edge = '#000099ff'
        curr_val = R[ds]['combined']['params'].loc[tr,label]
        plot_kde(ax, data, label, clr_face, clr_edge, curr_val)

        # Plot green parameters
        for pi, label in enumerate(pn_green):
            ax = plt.subplot(gs.new_subplotspec((0, pi+1)))
            data = par_kde['combined'][label]
            clr_face = '#00ff0055'
            clr_edge = '#009900ff'
            curr_val = R[ds]['combined']['params'].loc[tr,label]
            plot_kde(ax, data, label, clr_face, clr_edge, curr_val)

        # Plot red parameters
        for pi, label in enumerate(pn_red):
            ax = plt.subplot(gs.new_subplotspec((1, pi+1)))
            data = par_kde['combined'][label]
            clr_face = '#ff000055'
            clr_edge = '#990000ff'
            curr_val = R[ds]['combined']['params'].loc[tr,label]
            plot_kde(ax, data, label, clr_face, clr_edge, curr_val)

        # Adjust subplot layout
        gs.tight_layout(fig, pad=0, rect=(0.5, 0, 1, 1))

        # Create axes for fit
        gs_fit = GridSpec(1, 1)
        ax = fig.add_subplot(gs_fit[0])
        gs_fit.tight_layout(fig, pad=0, rect=(0, 0, 0.5, 1))

    else:
        ax = fig.gca()

    #wr = np.sqrt(D[ds]['rfp'][:,tr])
    #ax.fill_between(D[ds]['t'], D[ds]['rfp'][:,tr]-wr, D[ds]['rfp'][:,tr]+wr, color='#ff000033')
    #wg = np.sqrt(D[ds]['gfp'][:,tr])
    #ax.fill_between(D[ds]['t'], D[ds]['gfp'][:,tr]-wg, D[ds]['gfp'][:,tr]+wg, color='#00ff0033')

    p_tr = ax.axvline(R[ds]['combined']['params']['tr'][tr], label='RFP onset',
                       color='#ff0000', linewidth=.5, linestyle='--')
    p_tg = ax.axvline(R[ds]['combined']['params']['tg'][tr], label='GFP onset',
                      color='#00ff00', linewidth=.5, linestyle='--')
    p_fr, = ax.plot(D[ds]['t'], R[ds]['combined']['fit']['red'][tr], '-', label='RFP (fit)', color='#ff0000', linewidth=1)
    p_fg, = ax.plot(D[ds]['t'], R[ds]['combined']['fit']['green'][tr], '-', label='GFP (fit)', color='#00ff00', linewidth=1)
    p_dr, = ax.plot(D[ds]['t'], D[ds]['rfp'][:,tr], '-', label='RFP (measured)', color='#990000', linewidth=.5)
    p_dg, = ax.plot(D[ds]['t'], D[ds]['gfp'][:,tr], '-', label='GFP (measured)', color='#009900', linewidth=.5)

    # Format plot
    ax.set_xlabel('Time [h]')
    ax.set_ylabel('Fluorescence intensity [a.u.]')
    ax.set_title('{} {} [{}] #{:03d}\n(combined fit)'.format(
        D[ds]['sample'], D[ds]['condition'], D[ds]['measurement'], tr))
    ax.legend(handles=[p_dg, p_fg, p_tg, p_dr, p_fr, p_tr])

    # Write figure to pdf
    if pdf != None:
        pdf.savefig(fig, bbox_inches='tight')

    # Show and close figure
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Fit combined model
for ds in range(len(D)):
    R[ds]['combined']['fit'] = {'red': [], 'green': []}
    nTraces = np.shape(D[ds]['rfp'])[1]

    for tr in range(nTraces):
        print('Fitting „{}“ #{:03d}/{:03d}. '.format(getDataLabel(ds), tr, nTraces))#, end='')

        # Get the data for fitting
        data = np.stack([D[ds]['rfp'][:,tr], D[ds]['gfp'][:,tr]], axis=1)
        
        # Adjust parameter properties for onset time and offset
        combined_p.set('tr', max=D[ds]['t'][data[:,0].argmax()])
        combined_p.set('offr',
                       min=data[:10,0].min(),
                       max=data[:10,0].max(),
                       value=np.median(data[:10,0]))
        combined_p.set('tg', max=D[ds]['t'][data[:,1].argmax()])
        combined_p.set('offg',
                       min=data[:10,1].min(),
                       max=data[:10,1].max(),
                       value=np.median(data[:10,1]))

        # Get amplitude correction
        amp_red = data[:,0].max() - data[:,0].min()
        amp_green = data[:,1].max() - data[:,1].min()
        amp_correct = amp_red - amp_green

        # Fit the data
        wght = np.stack([D[ds]['rfp_error'][:,tr], D[ds]['gfp_error'][:,tr]], axis=1)**2

        # Set up objective function (as closure)
        #i_obj = 0
        #isChisqNan = False
        def objective_fcn(params):
            """Objective function for combined model"""

            # Compute chisquare
            cur_val = combined_p.eval(params, t=D[ds]['t'])
            chisq = np.sum(.5 * (data - cur_val)**2 / wght)

            # DEBUG
            #global i_obj, isChisqNan
            #print("Trace {:03d}, i={:03d}: chisq={}".format(tr, i_obj, chisq))
            #for p,v in zip(combined_p.names(), params):
            #    print("\t{:>10s} = {}".format(p, v))

            # Print model values at first iteration with NaN chisquare
            #if np.isnan(chisq) and not isChisqNan:
            #    print(cur_val)
            #    isChisqNan = True
            #i_obj += 1

            return chisq
        result = sc.optimize.minimize(objective_fcn,
                                      combined_p.initial(),
                                      method='TNC',#'L-BFGS-B','TNC'
                                      bounds=combined_p.bounds(),
                                      options={'disp': True,
                                               'maxiter': 10000})

        # Save results to R
        R[ds]['combined']['params'].iloc[tr] = result.x
        best_fit = combined(D[ds]['t'], *result.x)
        R[ds]['combined']['fit']['red'].insert(tr, best_fit[:,0])
        R[ds]['combined']['fit']['green'].insert(tr, best_fit[:,1])

        # Print result
        print("\tSuccess {}: {}".format(result.success, result.message))

        # DEBUG
        #if tr >= 40:
        #    print("Breaking loop for debugging purposes")
        #    break

In [ ]:
# Plot results of combined fit
ts = getTimeStamp()
for ds in range(len(D)):
    par_kde = {}
    for t in ('combined',):
        par_kde[t] = parameter_KDE(R[ds][t]['params'])
    pdffile = os.path.join(getOutpath(), '{}_combined_{}.pdf'.format(ts, getDataLabel(ds, True)))
    with PdfPages(pdffile) as pdf:
        for tr in range(np.shape(D[ds]['rfp'])[1]):
            plotCombined(ds, tr, pdf, par_kde=par_kde)
            
            # DEBUG
            #if tr >= 40:
            #    print("Forcing break")
            #    break

In [ ]:
# Plot violin distributions of the data sets (both separate and combined)
pn_both = ('m_ktl',)
pn_red = ('tr', 'kmr', 'betr', 'deltr', 'offr')
pn_green = ('tg', 'kmg', 'betg', 'deltg', 'offg')

grid = (2, len(pn_both)+max(len(pn_red), len(pn_green)))

with PdfPages(os.path.join(getOutpath(), '{:s}_parameter_distributions.pdf'.format(getTimeStamp()))) as pdf:
    for ds in range(len(D)):

        par_kde = {}
        fit_types = []

        # Check for separate fit
        if 'red' in R[ds] and 'green' in R[ds]:
            hasSeparate = True
            fit_types += ['red', 'green']
        else:
            hasSeparate = False

        # Check for combined fit
        par_kde_combined = {}
        if 'combined' in R[ds]:
            hasCombined = True
            fit_types += ['combined']
        else:
            hasCombined = False

        # Calculate parameter distributions
        for t in fit_types:
            par_kde[t] = parameter_KDE(R[ds][t]['params'])

        # Plot parameter distributions
        for typeName, hasType in zip(('separate', 'combined'), (hasSeparate, hasCombined)):
            if not hasType:
                continue

            fig = plt.figure()
            gs = GridSpec(grid[0], grid[1])

            if typeName == 'combined':
                # Combined fit; define specific settings
                pn_green_temp = pn_green
                pn_red_temp = pn_red
                offset_both = len(pn_both)
                kde_label_green = 'combined'
                kde_label_red = 'combined'

                # Plot combined parameters
                for pi, label in enumerate(pn_both):
                    ax = plt.subplot(gs.new_subplotspec((pi, 0), rowspan=2))
                    data = par_kde['combined'][label]
                    clr_face = '#0000ff55'
                    #clr_edge = '#000099ff'
                    plot_kde(ax, data, label, clr_face)
            else:
                # Separate fit; define specific settings
                pn_green_temp = pn_both + pn_green
                pn_red_temp = pn_both + pn_red
                offset_both = 0
                kde_label_green = 'green'
                kde_label_red = 'red'

            # Plot green parameters
            for pi, par_label in enumerate(pn_green_temp):
                ax = plt.subplot(gs.new_subplotspec((0, pi+offset_both)))
                data = par_kde[kde_label_green][par_label]
                clr_face = '#00ff0055'
                #clr_edge = '#009900ff'
                plot_kde(ax, data, par_label, clr_face)

            # Plot red parameters
            for pi, par_label in enumerate(pn_red_temp):
                ax = plt.subplot(gs.new_subplotspec((1, pi+offset_both)))
                data = par_kde[kde_label_red][par_label]
                clr_face = '#ff000055'
                #clr_edge = '#990000ff'
                plot_kde(ax, data, par_label, clr_face)

            # Show and close figure
            fig.suptitle(getDataLabel(ds) + " (" + typeName + " fit)")
            fig.tight_layout(pad=0, rect=(0, 0, 1, .93))
            pdf.savefig(fig, bbox_inches='tight')
            plt.show(fig)
            plt.close(fig)

## Playground
This section contains code that was/is used for developing ideas.

In [ ]:
# Plot the parameter distributions for the datasets
ds_keys = list(R.keys())
ds_keys.sort()
params = R[ds_keys[0]]['combined']['params'].columns
grid = (len(params), len(ds_keys))
i_col = 0

pdffile = os.path.join(getOutpath(), '{:s}_parameters.pdf'.format(getTimeStamp()))
with PdfPages(pdffile) as pdf:
    fig = plt.figure()
    fig.set_figheight(grid[0] * .8 * fig.get_figheight())
    fig.set_figwidth(grid[1] * .8 * fig.get_figwidth())

    for ds in ds_keys:
        i_row = 0
        for p in params:
            ax = plt.subplot2grid(grid, (i_row, i_col))
            ax.hist(R[ds]['combined']['params'][p], bins=100)
            if i_row == grid[0] - 1:
                ax.set_xlabel('Value [a.u.]')
            if i_col == 0:
                ax.set_ylabel('Occurrences [#]')
            ax.set_title('{:s}: {:s}'.format(ds, p))
            i_row += 1
        i_col += 1

    pdf.savefig(fig)
    plt.show(fig)
    plt.close(fig)

In [ ]:
# Plot onset time correlations
pdffile = os.path.join(getOutpath(), '{:s}_onset_correlations.pdf'.format(getTimeStamp()))
with PdfPages(pdffile) as pdf:
    for k in R.keys():
        fig = plt.figure()
        plt.plot([0, 30], [0, 30], 'k-')
        plt.plot(R[k]['combined']['params']['tr'], R[k]['combined']['params']['tg'], '.')
        plt.xlabel('Onset RFP [h]')
        plt.ylabel('Onset GFP [h]')
        plt.title(k)
        pdf.savefig(fig)
        plt.show()
        plt.close()
    

In [ ]:
# Degradation rate ratio
def plotHistograms(maxH):
    Rkeys = sorted(R.keys())
    for ds in Rkeys:
        #deltg = R[ds]['green']['params']['deltg']
        #deltr = R[ds]['red']['params']['deltr']
        deltg = R[ds]['combined']['params']['deltg']
        deltr = R[ds]['combined']['params']['deltr']
        quot = deltg / deltr

        fig = plt.figure()
        plt.hist(quot, bins=150, range=(0, maxH))
        plt.title(ds)
        plt.xlabel('$\delta_\mathrm{green} / \delta_\mathrm{red}$ [a.u.]')
        plt.ylabel('Occurrences [#]')
        plt.show(fig)
        plt.close(fig)

wdg.interact(plotHistograms, maxH=wdg.IntSlider(
    value=100, min=0, max=1000, step=10, description='Histogram maximum', continuous_update=False));

In [ ]:
# Fit distribution to degradation rate quotient histograms
def gamma(x, p=2, b=1, s=10):
    return s * b**p * x**(p-1) * np.exp(-b * x) / sc.special.gamma(p)

def gamma2(x, p1=1.9, p2=2.1, b1=0.9, b2=1.1, s1=10, s2=10):
    return gamma(x, p1, b1, s1) + gamma(x, p2, b2, s2)

def weibull(x, lmbd=.2, k=2, s=10):
    return s * lmbd * k * (lmbd * x)**(k - 1) * np.exp(- (lmbd * x)**k)

def weibull2(x, lmbd1=.15, lmbd2=.25, k1=1.9, k2=2.1, s1=10, s2=10):
    return weibull(x, lmbd=lmbd1, k=k1, s=s1) + weibull(x, lmbd=lmbd2, k=k2, s=s2)

# Define models
model_gamma = lm.Model(gamma)
model_gamma.set_param_hint(name='p', min=.01)
model_gamma.set_param_hint(name='b', min=.01)
model_gamma.set_param_hint(name='s', min=1)

model_gamma2 = lm.Model(gamma2)
model_gamma2.set_param_hint(name='p1', min=.01)
model_gamma2.set_param_hint(name='p2', min=.01)
model_gamma2.set_param_hint(name='b1', min=.01)
model_gamma2.set_param_hint(name='b2', min=.01)
model_gamma2.set_param_hint(name='s1', min=1)
model_gamma2.set_param_hint(name='s2', min=1)

model_weibull = lm.Model(weibull)
model_weibull.set_param_hint(name='lmbd', min=.001)
model_weibull.set_param_hint(name='k', min=.001, max=5)
model_weibull.set_param_hint(name='s', min=1)

model_weibull2 = lm.Model(weibull2)
model_weibull2.set_param_hint(name='lmbd1', min=.001)
model_weibull2.set_param_hint(name='lmbd2', min=.001)
model_weibull2.set_param_hint(name='k1', min=.001, max=5)
model_weibull2.set_param_hint(name='k2', min=.001, max=5)
model_weibull2.set_param_hint(name='s1', min=1)
model_weibull2.set_param_hint(name='s2', min=1)

maxH = 40

with PdfPages(os.path.join(getOutpath(), '{:s}_degradation_distribution.pdf'.format(getTimeStamp()))) as pdf:
    for ds in sorted(R.keys()):
        # Calculate degradation rate quotient
        deltg = R[ds]['combined']['params']['deltg']
        deltr = R[ds]['combined']['params']['deltr']
        quot = deltg / deltr

        # Create histogram
        fig = plt.figure()
        ax = fig.add_subplot(1, 2, 1)
        hist_val, hist_edg = ax.hist(quot, bins=70, range=(0, maxH), label='Histogram')[:2]
        hist_ctr = (hist_edg[:-1] + hist_edg[1:]) / 2

        # Fit models
        result_g = model_gamma.fit(hist_val, x=hist_ctr)
        result_g2 = model_gamma2.fit(hist_val, x=hist_ctr)
        result_w = model_weibull.fit(hist_val, x=hist_ctr)
        result_w2 = model_weibull2.fit(hist_val, x=hist_ctr)

        # Select models
        #print('gamma: {}'.format(result_g.chisqr))
        #print('gamma2: {}'.format(result_g2.chisqr))
        #print('weibull: {}'.format(result_w.chisqr))
        #print('weibull2: {}'.format(result_w2.chisqr))

        if result_g2.chisqr < .7 * result_g.chisqr:
            res_g = result_g2
            name_g = 'gamma2'
        else:
            res_g = result_g
            name_g = 'gamma'

        if result_w2.chisqr < .7 * result_w.chisqr:
            res_w = result_w2
            name_w = 'weibull2'
        else:
            res_w = result_w
            name_w = 'weibull'

        # Plot models
        x = np.linspace(.1, 5, 100)
        ax.plot(hist_ctr, res_g.best_fit, '-', label=name_g, color='orange')
        ax.plot(hist_ctr, res_w.best_fit, '-', label=name_w, color='magenta')
        ax.legend()
        ax.set_xlabel('$\delta_\mathrm{green} / \delta_\mathrm{red}$ [a.u.]')
        ax.set_ylabel('Counts [#]')
        ax.set_title(ds)

        # Print fit reports
        rep = res_g.fit_report(show_correl=False) + '\n' + res_w.fit_report(show_correl=False)
        ax = fig.add_subplot(1, 2, 2)
        ax.set_axis_off()
        ax.text(0, 1, rep, ha='left', va='top', family='monospace', size=5.5)

        # Display, save and close figure
        plt.show(fig)
        pdf.savefig(fig)
        plt.close(fig)

In [ ]:
# Scatter plot of degradation rates
Rkeys = sorted(R.keys())
for ds in Rkeys:
    deltg = R[ds]['combined']['params']['deltg']
    deltr = R[ds]['combined']['params']['deltr']

    fig = plt.figure()
    h = plt.plot(deltg, deltr, '.')
    plt.title(ds)
    plt.xlabel('$\delta_\mathrm{green}$ [a.u.]')
    plt.ylabel('$\delta_\mathrm{red}$ [a.u.]')
    plt.xscale('log')
    plt.yscale('log')
    plt.show(fig)
    plt.close(fig)